In [1]:
%reload_ext autoreload
%autoreload 2
import os
import glob
import scipy.io.wavfile as wv
import matplotlib.pyplot as plt
import numpy as np
from util import util
import scipy.signal as sig

Using TensorFlow backend.


In [24]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir

baseTargetDir = '/Users/etto/Desktop/pData'
baseTargetDir = 'E:/SAA_DATA/targetDir'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

NFFT = 1024

In [22]:
# Train alleen op 1 van de org dirs
srcDir = baseSrcDir + '/' + orgWavDirs1[0]
filesPerCategory = {}
for catDirLong in glob.glob(srcDir + '/*'):
    catDir = catDirLong.replace('\\', '/')
    catDir = catDir.replace(srcDir + '/', '')
    
    filesPerCategory[catDir] = []
    for filename in glob.glob(catDirLong + '/*'):
        filename = filename.replace('\\','/')
        filesPerCategory[catDir].append(filename)


In [25]:
# maak spectros van elke file
spectros = {}
for clz in classes:
    spectros[clz] = []
    for filename in filesPerCategory[clz]:
        fs, signal = wv.read(filename)
        freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
        spectros[clz].append(spectrogram.T)
        

In [42]:
#alle X achter elkaar in goede volgorde

X_train = np.concatenate(spectros[classes[0]], axis=0)
for i in range(1, len(classes)):
    nwSpectros = np.concatenate(spectros[classes[i]], axis=0)
    X_train = np.concatenate((X_train,nwSpectros), axis=0) 

np.shape(X_train)

(187316, 513)

In [36]:
clzLengths = {}
for clz in classes:
    clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectros[clz]])
print(clzLengths)

{'music': 60205, 'voice': 60859, 'environment': 66252}


In [82]:
# one-hot encoding
clsNr = 0
nrFiles = clzLengths[classes[0]]
Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

nrFiles = clzLengths[classes[1]]
Y_train_nw = np.array((np.zeros(nrFiles),np.ones(nrFiles), np.zeros(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

nrFiles = clzLengths[classes[2]]
Y_train_nw = np.array((np.zeros(nrFiles),np.zeros(nrFiles), np.ones(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
print(np.shape(Y_train))

(187316, 3)


zie https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/